## Asymptotic Analysis of LASSO Model Fit Quality

The LASSO regression model fits effective resistance $R_{\text{eff}}(u, v)$ as a function of geometric features. The key question we want to investigate is: **how does the quality of the model fit behave in the thermodynamic limit as $n \to \infty$?**

### Setup

We study fit quality (measured by out-of-sample $R^2$) across a grid of $(n, k)$ values, where:

- $n \in [256, 512, 1024, 2048, 4096]$ is the number of nodes
- $k \in [5, 6, 7, 8, 9]$ is the **expected mean degree**, which controls the connection radius via $r = \sqrt{k / ((n-1)\pi)}$

The model is trained by pooling samples from independently generated RGGs and evaluated on held-out graphs, ensuring the $R^2$ score reflects out-of-sample generalisation rather than in-sample memorisation.

### The Fixed Mean Degree Regime

In the **supercritical regime** used here, $k$ is held fixed as $n$ grows. Since the radius $r \propto n^{-1/2}$ shrinks as $n$ increases, each node's neighbourhood becomes a smaller and smaller fraction of the torus. Locally, the graph looks increasingly like an infinite planar random geometric graph.

This raises a natural question: does our fixed geometric feature set remain a good predictor of $R_{\text{eff}}$ in this super critical regime limit, or does the fit deteriorate as the graph grows large and the effective resistance landscape becomes harder to capture with low-order terms?

We expect the answer to depend on $k$:

The reference curve $\ln(n) = k$ marks the boundary of the **connected regime**: values of n above this curve make the supercritical radius no longer sufficient to guarantee connectivity almost surely. Points beyond this line should be interpreted with caution, as disconnected graphs are resolved by restricting to the largest connected component.

### What We Are Looking For

The plot produced below allows us to visually assess:

1. Whether $R^2$ is broadly stable across $n$ for a fixed $k$ (suggesting the feature set captures the right asymptotic structure), or whether it decays systematically.
2. Whether higher $k$ consistently yields better fits, and whether this effect strengthens or weakens as $n$ grows.
3. Whether there is a phase transition in fit quality near the $\ln(n) = k$ boundary.

In [ ]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt
import warnings
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from RGG_Library import RGGBuilder
import pandas as pd

SPACE = "torus"
USE_ANGLES = True
PERTURB = False

# ---------------------------
# GRID PARAMETERS
# ---------------------------
N_VALUES            = [512, 1024, 2048, 4096, 8192]
K_VALUES            = [5, 6, 7, 8, 10, 12, 16, 20]
NSAMPLES            = 1000
N_TRAIN_GRAPHS      = 30
N_TEST_GRAPHS       = 20
BASE_SEED           = 0

# ---------------------------
# HELPER: build graph & extract features
# ---------------------------
def build_features(n, k, nsamples, seed):
    """
    Build one RGG and return (X, y). Returns (None, None) on failure.
    """
    builder = RGGBuilder(
        n=n, k=k, connectivity_regime="sc",
        space=SPACE, order=4, perturb=PERTURB, seed=seed
    )
    G = builder.build()

    if not nx.is_connected(G):
        components = list(nx.connected_components(G))
        if not components:
            return None, None
        Gsub = G.subgraph(max(components, key=len)).copy()
    else:
        Gsub = G

    Gsub = nx.convert_node_labels_to_integers(Gsub, ordering="sorted")

    random.seed(seed)
    np.random.seed(seed)

    # Useful if one wants to ignore angular effects
    if USE_ANGLES:
        res, degs, dists, pairs, angles = RGGBuilder.sample_commute_times_even_distance_w_angles(
            Gsub, nsamples=nsamples, n_bins=20, seed=seed,
            min_dist=builder.radius, max_dist=2
        )
    else:
        res, degs, dists, pairs = RGGBuilder.sample_commute_times_even_distance(
            Gsub, nsamples=nsamples, n_bins=20, seed=seed,
            min_dist=builder.radius, max_dist=2
        )

    if len(res) < 10:
        return None, None

    safe_dists = dists.copy()
    safe_dists[safe_dists == 0] = 1e-9
    current_angles = angles if USE_ANGLES else np.zeros_like(dists)

    X = np.column_stack([
        dists**2,
        np.log(safe_dists),
        dists**4 * np.cos(4 * current_angles),
        degs,
    ])
    y = np.array(res)

    return X, y


# ---------------------------
# CORE FUNCTION
# ---------------------------
def compute_r2(n, k, nsamples=NSAMPLES, base_seed=BASE_SEED):
    """
    Train LASSO on n_train_graphs pooled graphs, test on n_test_graphs pooled graphs.
    Returns out-of-sample R². Returns NaN on failure.
    """
    try:
        # --- TRAINING: pool data from n_train_graphs graphs ---
        X_train_parts, y_train_parts = [], []
        for i in range(N_TRAIN_GRAPHS):
            X, y = build_features(n, k, nsamples, seed=base_seed + i)
            if X is not None:
                X_train_parts.append(X)
                y_train_parts.append(y)

        if len(X_train_parts) == 0:
            return np.nan

        X_train = np.vstack(X_train_parts)
        y_train = np.concatenate(y_train_parts)

        # --- FIT LASSO ---
        model = make_pipeline(
            StandardScaler(),
            Lasso(alpha=1e-6, random_state=0, max_iter=50000)
        )
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(X_train, y_train)

        lasso  = model.named_steps['lasso']
        scaler = model.named_steps['standardscaler']
        real_coefs     = lasso.coef_ / scaler.scale_
        real_intercept = lasso.intercept_ - np.sum(real_coefs * scaler.mean_)

        # --- TESTING: pool data from n_test_graphs new graphs ---
        X_test_parts, y_test_parts = [], []
        for i in range(N_TEST_GRAPHS):
            # Use seeds that don't overlap with training seeds
            X, y = build_features(n, k, nsamples, seed=base_seed + N_TRAIN_GRAPHS + i)
            if X is not None:
                X_test_parts.append(X)
                y_test_parts.append(y)

        if len(X_test_parts) == 0:
            return np.nan

        X_test = np.vstack(X_test_parts)
        y_test = np.concatenate(y_test_parts)

        # --- OUT-OF-SAMPLE R² ---
        y_pred  = X_test @ real_coefs + real_intercept
        ss_res  = np.sum((y_test - y_pred) ** 2)
        ss_tot  = np.sum((y_test - np.mean(y_test)) ** 2)
        r2      = 1 - ss_res / ss_tot

        return float(r2)

    except Exception as e:
        print(f"  [WARN] n={n}, k={k} failed: {e}")
        return np.nan


# ---------------------------
# SWEEP OVER GRID
# ---------------------------
R2_grid = np.full((len(N_VALUES), len(K_VALUES)), np.nan)

total = len(N_VALUES) * len(K_VALUES)
done  = 0

for i, n in enumerate(N_VALUES):
    for j, k in enumerate(K_VALUES):
        done += 1
        print(f"[{done}/{total}] Running n={n}, k={k} ...", end="  ")
        r2 = compute_r2(n, k, nsamples=NSAMPLES, base_seed=BASE_SEED)
        R2_grid[i, j] = r2
        print(f"R² = {r2:.4f}" if not np.isnan(r2) else "R² = NaN")

# ---------------------------
# BUILD DATAFRAME
# ---------------------------
records = []
for i, n in enumerate(N_VALUES):
    for j, k in enumerate(K_VALUES):
        records.append({
            "n":         n,
            "k":         k,
            "ln(n)":     np.log(n),
            "r2":        R2_grid[i, j],
            "above_threshold": k >= np.log(n),   # whether k is above ln(n)
        })

df = pd.DataFrame(records)
df["r2"] = df["r2"].astype(float)

print("\nResults DataFrame:")
print(df.to_string(index=False))

CSV_PATH = "Assymptotic_Analysis_R2_Results.csv"
df.to_csv(CSV_PATH, index=False)
print(f"\nDataFrame saved to {CSV_PATH}")

# ---------------------------
# LINE PLOT  (R² vs k, coloured by n)
# ---------------------------

# Organise df into dict: r2_dict[n][k] = r2
r2_dict = {}
for _, row in df.iterrows():
    n, k, r2_val = int(row["n"]), row["k"], row["r2"]
    if n not in r2_dict:
        r2_dict[n] = {}
    r2_dict[n][k] = r2_val

cmap = plt.cm.plasma
norm = plt.Normalize(vmin=np.log2(min(N_VALUES)), vmax=np.log2(max(N_VALUES)))

fig, ax = plt.subplots(figsize=(9, 6))

for n in sorted(r2_dict.keys()):
    ks   = sorted(r2_dict[n].keys())
    vals = [r2_dict[n][k] for k in ks]
    color = cmap(norm(np.log2(n)))

    # Separate valid and NaN points so the line isn't broken by NaNs
    ks_arr   = np.array(ks, dtype=float)
    vals_arr = np.array(vals, dtype=float)
    valid    = ~np.isnan(vals_arr)

    ax.plot(ks_arr[valid], vals_arr[valid],
            marker='o', linewidth=2, markersize=6,
            color=color, label=f"n = {n:,}")

    # Annotate each valid point with its R² value
    for k_pt, v_pt in zip(ks_arr[valid], vals_arr[valid]):
        ax.annotate(
            f"{v_pt:.3f}", (k_pt, v_pt),
            textcoords="offset points", xytext=(0, 8),
            ha='center', fontsize=7, color=color, fontweight='bold'
        )

    # Mark NaN points with grey X
    if np.any(~valid):
        ax.scatter(ks_arr[~valid], np.zeros_like(ks_arr[~valid]),
                   c='lightgrey', marker='X', s=150,
                   edgecolors='k', linewidths=0.6, zorder=3)

# Vertical dotted lines at k = ln(n) for each n
for n in sorted(r2_dict.keys()):
    color = cmap(norm(np.log2(n)))
    ax.axvline(np.log(n), color=color, linewidth=0.8, linestyle=':', alpha=0.7)

# Colorbar keyed to n
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.02)
cbar.set_label(r"$\log_2(n)$", fontsize=12)
cbar.set_ticks([np.log2(n) for n in sorted(r2_dict.keys())])
cbar.set_ticklabels([f"{n:,}" for n in sorted(r2_dict.keys())])

ax.set_xlabel("Mean Degree  $k$", fontsize=13)
ax.set_ylabel("Out-of-Sample  $R^2$", fontsize=13)
ax.set_title(
    f"LASSO $R^2$ vs Mean Degree across System Sizes\n"
    f"(Train: {N_TRAIN_GRAPHS} graphs × {NSAMPLES} pts  |  Test: {N_TEST_GRAPHS} graphs × {NSAMPLES} pts)",
    fontsize=12
)
ax.set_xticks(K_VALUES)
ax.set_ylim(0, 1.05)
ax.axhline(1.0, color='k', linewidth=0.8, linestyle='--', alpha=0.3)
ax.legend(fontsize=9)
ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig("r2_vs_k.png", dpi=150)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------
# DATA OBTAINED FROM PREVIOUS CELLS
# ------------------------------------------------------
data = [
    (512,  5, 0.2108), (512,  6, 0.3565), (512,  7, 0.6125), (512,  8, 0.7051), (512,  9, 0.7830),
    (1024, 5, 0.2142), (1024, 6, 0.3649), (1024, 7, 0.5389), (1024, 8, 0.6144), (1024, 9, 0.7130),
    (2048, 5, 0.2101), (2048, 6, 0.3681), (2048, 7, 0.5149), (2048, 8, 0.6256), (2048, 9, 0.7141),
    (4096, 5, 0.2043), (4096, 6, 0.3880), (4096, 7, 0.5375), (4096, 8, 0.6927), (4096, 9, 0.7730),
]

n_values = sorted(set(d[0] for d in data))
k_values = sorted(set(d[1] for d in data))

# Organise into dict: r2[n][k] = r2
r2 = {n: {} for n in n_values}
for n, k, val in data:
    r2[n][k] = val

# ------------------------------------------------------
# PLOT
# ------------------------------------------------------
cmap = plt.cm.plasma
norm = plt.Normalize(vmin=np.log2(min(n_values)), vmax=np.log2(max(n_values)))

fig, ax = plt.subplots(figsize=(8, 5))

for n in n_values:
    ks   = sorted(r2[n].keys())
    vals = [r2[n][k] for k in ks]
    color = cmap(norm(np.log2(n)))
    ax.plot(ks, vals, marker='o', linewidth=2, markersize=6, color=color, label=f"n = {n:,}")

# Vertical lines at k = ln(n) for each n
for n in n_values:
    color = cmap(norm(np.log2(n)))
    ax.axvline(np.log(n), color=color, linewidth=0.8, linestyle=':', alpha=0.7)

# Colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.02)
cbar.set_label(r"$\log_2(n)$", fontsize=12)
cbar.set_ticks([np.log2(n) for n in n_values])
cbar.set_ticklabels([f"{n:,}" for n in n_values])

ax.set_xlabel("Mean Degree  $k$", fontsize=13)
ax.set_ylabel("Out-of-Sample  $R^2$", fontsize=13)
ax.set_title("LASSO $R^2$ vs Mean Degree across System Sizes", fontsize=13)
ax.set_xticks(k_values)
ax.set_ylim(0, 1)
ax.axhline(1.0, color='k', linewidth=0.8, linestyle='--', alpha=0.3)
ax.legend(fontsize=9)
ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig("r2_vs_k.png", dpi=150)
plt.show()